In [22]:
import plotly.express as px
import polars as pl
from itables import show
import altair as alt

data_pus = pl.read_csv("data/data_pus.csv")
data_mix = pl.read_csv("data/data_mix_kontra.csv")

def bulan_hingga(bulan_terpilih):
    daftar_bulan = ["JANUARI", "FEBRUARI", "MARET", "APRIL", "MEI", "JUNI", "JULI", "AGUSTUS", "SEPTEMBER", "OKTOBER", "NOVEMBER", "DESEMBER"]
    index = daftar_bulan.index(bulan_terpilih)
    return daftar_bulan[:index + 1]

filter_kabupaten = ["PASANGKAYU", "PASANGKAYU", "MAMUJU", "MAMUJU"]
filter_kecamatan = ["PASANGKAYU", "PASANGKAYU", "PAPALANG", "PAPALANG"]
filter_desa = ["AKO", "GUNUNG SARI", "BODA-BODA", "SISANGO"]
filter_bulan = 'FEBRUARI'

# Urutan bulan yang benar
month_order = {
    "JANUARI": 1,
    "FEBRUARI": 2,
    "MARET": 3,
    "APRIL": 4,
    "MEI": 5,
    "JUNI": 6,
    "JULI": 7,
    "AGUSTUS": 8,
    "SEPTEMBER": 9,
    "OKTOBER": 10,
    "NOVEMBER": 11,
    "DESEMBER": 12
}


warna_kuning = "#d4a017"

warna_biru = "#3498db"


In [3]:
show(
    data_pus.filter(
        pl.col("KABUPATEN").is_in(filter_kabupaten),
        pl.col("KECAMATAN").is_in(filter_kecamatan),
        pl.col("KELURAHAN").is_in(filter_desa),
        pl.col("BULAN").is_in(bulan_hingga(filter_bulan))
    ).group_by(
        'PROVINSI', 'BULAN'
    ).agg([
        pl.col("PUS").sum()
    ])
)

In [4]:
aggregated =  data_pus.filter(
        pl.col("KABUPATEN").is_in(filter_kabupaten),
        pl.col("KECAMATAN").is_in(filter_kecamatan),
        pl.col("KELURAHAN").is_in(filter_desa),
        pl.col("BULAN").is_in(bulan_hingga(filter_bulan))
    ).group_by(
        'PROVINSI', 'BULAN'
    ).agg([
        pl.col("PUS").sum()
    ])

aggregated

PROVINSI,BULAN,PUS
str,str,i64
"""SULAWESI BARAT""","""FEBRUARI""",1786
"""SULAWESI BARAT""","""JANUARI""",1769


In [14]:
# Urutan bulan yang benar
month_order = {
    "JANUARI": 1,
    "FEBRUARI": 2,
    "MARET": 3,
    "APRIL": 4,
    "MEI": 5,
    "JUNI": 6,
    "JULI": 7,
    "AGUSTUS": 8,
    "SEPTEMBER": 9,
    "OKTOBER": 10,
    "NOVEMBER": 11,
    "DESEMBER": 12
}

# Proses data dengan Polars
df_processed = (
    aggregated
    .with_columns(
        pl.col("BULAN").replace(bulan_order).alias("month_order"),
        # Format angka dengan pemisah titik (.) untuk tooltip
        pl.col("PUS").map_elements(
            lambda x: f"{x:,}".replace(",", "."),
            return_dtype=pl.Utf8
        ).alias("PUS_formatted")
    )
    .sort("month_order")
)

# Hitung batas y-axis
min_value = df_processed["PUS"].min()
max_value = df_processed["PUS"].max()
y_min = min_value - (min_value * 0.05)
y_max = max_value + (max_value * 0.05)

# Konversi ke Pandas untuk Altair
df_for_viz = df_processed.to_pandas()

# Buat grafik
chart = alt.Chart(df_for_viz).mark_line(point=True).encode(
    x=alt.X('BULAN:N', 
            sort=list(bulan_order.keys()),
            axis=alt.Axis(
                labelAngle=0,  # Label horizontal
                labelLimit=150,  # Batas lebar label sebelum otomatis rotate
                labelOverlap="parity"  # Hindari tumpang tindih
            )),
    y=alt.Y('PUS:Q',
            scale=alt.Scale(domain=(y_min, y_max), nice=False),
            axis=alt.Axis(
                labelExpr='replace(format(datum.value, ",.0f"), ",", ".")',
                title='Jumlah PUS'
            )),
    tooltip=[
        alt.Tooltip('BULAN:N', title='Bulan'),
        alt.Tooltip('PUS_formatted:N', title='Jumlah PUS')
    ]
).properties(
    title='Tren Jumlah PUS di Sulawesi Barat', width = 600, height = 400
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False
).configure_point(
    size=100
).configure(
    padding=20,  # Tambah padding
    background='#f6f8fa'  # Warna background
)

chart

alt.Chart(...)

In [30]:

# Buat anotasi untuk titik awal
start_label = alt.Chart(df_for_viz).transform_filter(
    alt.datum.BULAN == df_for_viz['BULAN'].min()
).mark_text(
    align='center',
    dy=-20,          # Geser ke atas titik
    fontSize=14,
    fontWeight='bold',
    color='black',    # WARNA HITAM
).encode(
    x=alt.X('BULAN:N', sort=list(month_order.keys())),
    y=alt.Y('PUS:Q'),
    text=alt.Text('PUS_formatted:N')
)

# Buat anotasi untuk titik akhir
end_label = alt.Chart(df_for_viz).transform_filter(
    alt.datum.BULAN == df_for_viz['BULAN'].max()
).mark_text(
    align='center',
    dy=-20, 
    fontSize=14,
    fontWeight='bold',
    color='black',    # WARNA HITAM
).encode(
    x=alt.X('BULAN:N', sort=list(month_order.keys())),
    y=alt.Y('PUS:Q'),
    text=alt.Text('PUS_formatted:N')
)

# Gabungkan semua elemen
chart = alt.layer(
    # Base chart (garis dan point)
    alt.Chart(df_for_viz).mark_line(
        point=alt.OverlayMarkDef(
            size=200,
            filled=True,
            fillOpacity=1,
            color='#3498db'  # Warna point biru
        )
    ).encode(
        x=alt.X('BULAN:N', 
                sort=list(month_order.keys()),
                axis=alt.Axis(
                    labelAngle=0,
                    labelLimit=150,
                    labelOverlap="parity"
                )),
        y=alt.Y('PUS:Q',
                scale=alt.Scale(domain=(y_min, y_max), nice=False),
                axis=alt.Axis(
                    labelExpr='replace(format(datum.value, ",.0f"), ",", ".")',
                    title='Jumlah PUS'
                )),
        tooltip=[
            alt.Tooltip('BULAN:N', title='Bulan'),
            alt.Tooltip('PUS_formatted:N', title='Jumlah PUS')
        ]
    ),
    
    # Anotasi
    start_label,
    end_label
).properties(
    title='Tren Jumlah PUS di Sulawesi Barat', width = 600, height = 400
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False
).configure(
    padding=20,
    background='#f6f8fa'
)

chart

alt.LayerChart(...)

In [35]:
alt.datum.BULAN == df_for_viz['BULAN'].max()

(datum.BULAN === 'SEPTEMBER')

# PA

In [4]:
aggregated =  data_mix.filter(
        pl.col("KABUPATEN").is_in(filter_kabupaten),
        pl.col("KECAMATAN").is_in(filter_kecamatan),
        pl.col("KELURAHAN").is_in(filter_desa),
        pl.col("BULAN").is_in(bulan_hingga(filter_bulan))
    ).group_by(
        'PROVINSI', 'BULAN'
    ).agg([
        pl.col("PA").sum()
    ])

aggregated

PROVINSI,BULAN,PA
str,str,i64
"""SULAWESI BARAT""","""FEBRUARI""",1293
"""SULAWESI BARAT""","""JANUARI""",1115


In [ ]:
df_processed =  data_mix.filter(
    pl.col("KABUPATEN").is_in(filter_kabupaten),
    pl.col("KECAMATAN").is_in(filter_kecamatan),
    pl.col("KELURAHAN").is_in(filter_desa),
    pl.col("BULAN").is_in(bulan_hingga(filter_bulan))
).group_by(
    'PROVINSI', 'BULAN'
).agg([
    pl.col("PA").sum()
])

# Proses data dengan Polars
df_processed = (
    df_processed
    .with_columns(
        # Konversi bulan ke numerik
        pl.col("BULAN")
        .replace(month_order)
        .cast(pl.Int32)
        .alias("month_order")
    )
    .with_columns(
        pl.col("PA")
        .map_elements(
            lambda x: f"{x:,}".replace(",", "."),
            return_dtype=pl.Utf8
        )
        .alias("JUMLAH PA")
    )
    .sort("month_order")
    .drop("month_order")
)

# Ambil bulan pertama dan terakhir
first_month = df_processed["BULAN"][0]
last_month = df_processed["BULAN"][-1]

# Hitung batas y-axis
y_min = df_processed["PA"].min() * 0.95
y_max = df_processed["PA"].max() * 1.05


In [ ]:
# === Base Chart ===
base = alt.Chart(df_processed).mark_line(
    point=alt.OverlayMarkDef(size=200, filled=True, fillOpacity=1, color='#3498db')
).encode(
    x=alt.X('BULAN:N', 
            sort=list(month_order.keys()),
            axis=alt.Axis(
                labelAngle=0, 
                labelLimit=200, 
                labelOverlap="parity",
                labelExpr="slice(datum.label, 0, 3)"  # Ambil 3 huruf pertama
            )),
    y=alt.Y('PA:Q',
            scale=alt.Scale(domain=(y_min, y_max)),
            axis=alt.Axis(
                labelExpr='replace(format(datum.value, ",.0f"), ",", ".")', 
                title='JUMLAH PA',
                tickCount=5  # BATASI 5 TICK SAJA
            )),
    tooltip=[alt.Tooltip('BULAN:N'), alt.Tooltip('JUMLAH PA:N')]
)

# === Anotasi Label (Perubahan warna) ===
start_label = alt.Chart(df_processed).transform_filter(
    alt.datum.BULAN == first_month
).mark_text(
    align='center', 
    dy=-25, 
    fontSize=11, 
    fontWeight='bold', 
    color='#3498db'  # Sesuaikan dengan warna point
).encode(
    x=alt.X('BULAN:N', sort=list(month_order.keys())),
    y=alt.Y('PA:Q'),
    text=alt.Text('JUMLAH PA:N')
)

end_label = alt.Chart(df_processed).transform_filter(
    alt.datum.BULAN == last_month
).mark_text(
    align='center', 
    dy=-25, 
    fontSize=11, 
    fontWeight='bold', 
    color='#3498db'  # Sesuaikan dengan warna point
).encode(
    x=alt.X('BULAN:N', sort=list(month_order.keys())),
    y=alt.Y('PA:Q'),
    text=alt.Text('JUMLAH PA:N')
)


chart = alt.layer(base, start_label, end_label).properties(
    title=alt.TitleParams(
        'Tren Jumlah PA',
        anchor='middle',  # Judul rata kiri
        offset=20
    ),
    width=600,  # Lebar menyesuaikan layar
    height=400
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=14
).configure(
    padding={"left": 0, "right": 0, "top": 20, "bottom": 0},  # Sesuaikan padding
    background='#f6f8fa',
    autosize=alt.AutoSizeParams(
        type='fit',
        contains='padding'
    )
).configure_legend(
    disable=True  # Hapus legenda jika tidak diperlukan
)
chart

alt.LayerChart(...)

# Unmet Need

In [37]:
df_processed = data_pus.filter(
    pl.col("KABUPATEN").is_in(filter_kabupaten),
    pl.col("KECAMATAN").is_in(filter_kecamatan),
    pl.col("KELURAHAN").is_in(filter_desa),
    pl.col("BULAN").is_in(bulan_hingga(filter_bulan))
).group_by('PROVINSI', 'BULAN').agg([
    pl.col("PUS").sum(),
    pl.col("UNMET NEED").sum()
]).with_columns(
    ((pl.col("UNMET NEED") / pl.col("PUS"))* 100).round(2).alias('UNMET NEED')
).with_columns(
    (pl.col("UNMET NEED").cast(pl.Utf8) + "%").alias("UNMET NEED (%)")  # Menambahkan kolom LABEL_UNMET_NEED
)
df_processed

PROVINSI,BULAN,PUS,UNMET NEED,UNMET NEED (%)
str,str,i64,f64,str
"""SULAWESI BARAT""","""JANUARI""",1769,8.76,"""8.76%"""
"""SULAWESI BARAT""","""FEBRUARI""",1786,8.68,"""8.68%"""


In [38]:
# Proses data dengan Polars
df_processed = (
    df_processed
    .with_columns(
        # Konversi bulan ke numerik
        pl.col("BULAN")
        .replace(month_order)
        .cast(pl.Int32)
        .alias("month_order")
    )
    .sort("month_order")
    .drop("month_order")
)

# Ambil bulan pertama dan terakhir
first_month = df_processed["BULAN"][0]
last_month = df_processed["BULAN"][-1]

# Hitung batas y-axis
y_min = df_processed["UNMET NEED"].min() * 0.95
y_max = df_processed["UNMET NEED"].max() * 1.05

df_processed

PROVINSI,BULAN,PUS,UNMET NEED,UNMET NEED (%)
str,str,i64,f64,str
"""SULAWESI BARAT""","""JANUARI""",1769,8.76,"""8.76%"""
"""SULAWESI BARAT""","""FEBRUARI""",1786,8.68,"""8.68%"""


In [50]:
# === Base Chart ===
base = alt.Chart(df_processed).mark_line(
    point=alt.OverlayMarkDef(size=200, filled=True, fillOpacity=1, color='#3498db')
).encode(
    x=alt.X('BULAN:N', 
            sort=list(month_order.keys()),
            axis=alt.Axis(
                labelAngle=0, 
                labelLimit=200, 
                labelOverlap="parity",
                labelExpr="slice(datum.label, 0, 3)"  # Ambil 3 huruf pertama
            )),
    y=alt.Y('UNMET NEED:Q',
            scale=alt.Scale(domain=(y_min, y_max)),
            axis=alt.Axis(
                title='PERSENTASE UNMET NEED',
                tickCount=3,  # BATASI 5 TICK SAJA
                )
            ),
    tooltip=[alt.Tooltip('BULAN:N'), alt.Tooltip('UNMET NEED (%):N')]
)

# === Anotasi Label (Perubahan warna) ===
start_label = alt.Chart(df_processed).transform_filter(
    alt.datum.BULAN == first_month
).mark_text(
    align='center', 
    dy=-25, 
    fontSize=11, 
    fontWeight='bold', 
    color='#3498db'  # Sesuaikan dengan warna point
).encode(
    x=alt.X('BULAN:N', sort=list(month_order.keys())),
    y=alt.Y('UNMET NEED:Q'),
    text=alt.Text('UNMET NEED (%):N')
)

end_label = alt.Chart(df_processed).transform_filter(
    alt.datum.BULAN == last_month
).mark_text(
    align='center', 
    dy=-25, 
    fontSize=11, 
    fontWeight='bold', 
    color='#3498db'  # Sesuaikan dengan warna point
).encode(
    x=alt.X('BULAN:N', sort=list(month_order.keys())),
    y=alt.Y('UNMET NEED:Q'),
    text=alt.Text('UNMET NEED (%):N')
)


chart = alt.layer(base, start_label, end_label).properties(
    title=alt.TitleParams(
        'Tren Persentase Unmet Need',
        anchor='middle',  # Judul rata kiri
        offset=20
    ),
    width=600,  # Lebar menyesuaikan layar
    height=400
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=14
).configure(
    padding={"left": 0, "right": 0, "top": 20, "bottom": 0},  # Sesuaikan padding
    background='#f6f8fa',
    autosize=alt.AutoSizeParams(
        type='fit',
        contains='padding'
    )
).configure_legend(
    disable=True  # Hapus legenda jika tidak diperlukan
)
chart

alt.LayerChart(...)

# MKJP

In [5]:
df_processed = data_mix.filter(
    pl.col("KABUPATEN").is_in(filter_kabupaten),
    pl.col("KECAMATAN").is_in(filter_kecamatan),
    pl.col("KELURAHAN").is_in(filter_desa),
    pl.col("BULAN").is_in(bulan_hingga(filter_bulan))
).group_by('PROVINSI', "BULAN").agg([
    pl.col("IMPLAN").sum(),
    pl.col("IUD").sum(),
    pl.col("VASEKTOMI").sum(),
    pl.col("TUBEKTOMI").sum(),
    pl.col("KB MODERN").sum()
]).with_columns(
    (((pl.col("IUD") + pl.col("IMPLAN") + pl.col("VASEKTOMI") + pl.col("TUBEKTOMI")) / pl.col("KB MODERN"))* 100).round(2).alias('MKJP')
).with_columns(
    (pl.col("MKJP").cast(pl.Utf8) + "%").alias("MKJP (%)")  # Menambahkan kolom LABEL_UNMET_NEED
)
df_processed

PROVINSI,BULAN,IMPLAN,IUD,VASEKTOMI,TUBEKTOMI,KB MODERN,MKJP,MKJP (%)
str,str,i64,i64,i64,i64,i64,f64,str
"""SULAWESI BARAT""","""JANUARI""",186,56,0,41,1088,26.01,"""26.01%"""
"""SULAWESI BARAT""","""FEBRUARI""",214,63,0,41,1272,25.0,"""25.0%"""


In [6]:
# Proses data dengan Polars
df_processed = (
    df_processed
    .with_columns(
        # Konversi bulan ke numerik
        pl.col("BULAN")
        .replace(month_order)
        .cast(pl.Int32)
        .alias("month_order")
    )
    .sort("month_order")
    .drop("month_order")
)

# Ambil bulan pertama dan terakhir
first_month = df_processed["BULAN"][0]
last_month = df_processed["BULAN"][-1]

# Hitung batas y-axis
y_min = df_processed["MKJP"].min() * 0.95
y_max = df_processed["MKJP"].max() * 1.05

df_processed

PROVINSI,BULAN,IMPLAN,IUD,VASEKTOMI,TUBEKTOMI,KB MODERN,MKJP,MKJP (%)
str,str,i64,i64,i64,i64,i64,f64,str
"""SULAWESI BARAT""","""JANUARI""",186,56,0,41,1088,26.01,"""26.01%"""
"""SULAWESI BARAT""","""FEBRUARI""",214,63,0,41,1272,25.0,"""25.0%"""


In [7]:
# === Base Chart ===
base = alt.Chart(df_processed).mark_line(
    point=alt.OverlayMarkDef(size=200, filled=True, fillOpacity=1, color='#3498db')
).encode(
    x=alt.X('BULAN:N', 
            sort=list(month_order.keys()),
            axis=alt.Axis(
                labelAngle=0, 
                labelLimit=200, 
                labelOverlap="parity",
                labelExpr="slice(datum.label, 0, 3)"  # Ambil 3 huruf pertama
            )),
    y=alt.Y('MKJP:Q',
            scale=alt.Scale(domain=(y_min, y_max)),
            axis=alt.Axis(
                title='PERSENTASE MKJP',
                tickCount=3,  # BATASI 5 TICK SAJA
                )
            ),
    tooltip=[alt.Tooltip('BULAN:N'), alt.Tooltip('MKJP (%):N')]
)

# === Anotasi Label (Perubahan warna) ===
start_label = alt.Chart(df_processed).transform_filter(
    alt.datum.BULAN == first_month
).mark_text(
    align='center', 
    dy=-25, 
    fontSize=11, 
    fontWeight='bold', 
    color='#3498db'  # Sesuaikan dengan warna point
).encode(
    x=alt.X('BULAN:N', sort=list(month_order.keys())),
    y=alt.Y('MKJP:Q'),
    text=alt.Text('MKJP (%):N')
)

end_label = alt.Chart(df_processed).transform_filter(
    alt.datum.BULAN == last_month
).mark_text(
    align='center', 
    dy=-25, 
    fontSize=11, 
    fontWeight='bold', 
    color='#3498db'  # Sesuaikan dengan warna point
).encode(
    x=alt.X('BULAN:N', sort=list(month_order.keys())),
    y=alt.Y('MKJP:Q'),
    text=alt.Text('MKJP (%):N')
)


chart = alt.layer(base, start_label, end_label).properties(
    title=alt.TitleParams(
        'Tren Persentase MKJP',
        anchor='middle',  # Judul rata kiri
        offset=20
    ),
    width=600,  # Lebar menyesuaikan layar
    height=400
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=14
).configure(
    padding={"left": 0, "right": 0, "top": 20, "bottom": 0},  # Sesuaikan padding
    background='#f6f8fa',
    autosize=alt.AutoSizeParams(
        type='fit',
        contains='padding'
    )
).configure_legend(
    disable=True  # Hapus legenda jika tidak diperlukan
)
chart

alt.LayerChart(...)

# Bar Mix Kontra

In [11]:
mix_kontra = data_mix.filter(
    pl.col("KABUPATEN").is_in(filter_kabupaten),
    pl.col("KECAMATAN").is_in(filter_kecamatan),
    pl.col("KELURAHAN").is_in(filter_desa),
    pl.col("BULAN").is_in([filter_bulan])
).group_by('PROVINSI').agg([
    pl.col("SUNTIK").sum(),
    pl.col("PIL").sum(),
    pl.col("KONDOM").sum(),
    pl.col("MAL").sum(),
    pl.col("IMPLAN").sum(),
    pl.col("IUD").sum(),
    pl.col("VASEKTOMI").sum(),
    pl.col("TUBEKTOMI").sum()
])

mix_kontra

PROVINSI,SUNTIK,PIL,KONDOM,MAL,IMPLAN,IUD,VASEKTOMI,TUBEKTOMI
str,i64,i64,i64,i64,i64,i64,i64,i64
"""SULAWESI BARAT""",664,278,12,0,214,63,0,41


In [12]:
# Ubah data menjadi format "long" menggunakan `unpivot`
mix_kontra = mix_kontra.unpivot(
    index="PROVINSI",  # Kolom yang tetap
    on=["SUNTIK", "PIL", "KONDOM", "MAL", "IMPLAN", "IUD", "VASEKTOMI", "TUBEKTOMI"],  # Kolom yang di-unpivot
    variable_name="METODE_KB",  # Nama kolom untuk metode KB
    value_name="JUMLAH"  # Nama kolom untuk jumlah pengguna
)

# Urutkan data berdasarkan jumlah pengguna (terbesar ke terkecil)
mix_kontra = mix_kontra.sort("JUMLAH", descending=True)
mix_kontra

PROVINSI,METODE_KB,JUMLAH
str,str,i64
"""SULAWESI BARAT""","""SUNTIK""",664
"""SULAWESI BARAT""","""PIL""",278
"""SULAWESI BARAT""","""IMPLAN""",214
"""SULAWESI BARAT""","""IUD""",63
"""SULAWESI BARAT""","""TUBEKTOMI""",41
"""SULAWESI BARAT""","""KONDOM""",12
"""SULAWESI BARAT""","""MAL""",0
"""SULAWESI BARAT""","""VASEKTOMI""",0


In [32]:
# Menghitung total jumlah pengguna
total_jumlah = mix_kontra["JUMLAH"].sum()

# Menambahkan kolom persentase (dengan 2 angka desimal)
mix_kontra = mix_kontra.with_columns(
    (pl.col("JUMLAH") / total_jumlah * 100).round(2).alias("PERSENTASE")
)

# Mengurutkan data berdasarkan jumlah secara descending
mix_kontra = mix_kontra.sort("JUMLAH", descending=True)

# Membuat grafik batang horizontal dengan Altair
chart = (
    alt.Chart(mix_kontra)
    .mark_bar(color=warna_biru)
    .encode(
        y=alt.Y("METODE_KB:N", title="Metode KB", sort="-x"),  # Sumbu Y: Metode KB, diurutkan berdasarkan jumlah
        x=alt.X("JUMLAH:Q", title="Jumlah Pengguna"),         # Sumbu X: Jumlah pengguna
        tooltip=[
            alt.Tooltip("METODE_KB", title="Metode KB"),
            alt.Tooltip("JUMLAH", title="Jumlah"),
            alt.Tooltip("PERSENTASE", title="Persentase (%)", format=".2f")  # Format 2 angka desimal
        ]
    )
    .properties(
        width=600,                                            # Lebar grafik
        height=300,                                           # Tinggi grafik
        title="Jumlah Pengguna Metode KB di Sulawesi Barat"   # Judul grafik
    )
)

# Menambahkan label di ujung batang
text = (
    alt.Chart(sorted_df)
    .mark_text(align="left", baseline="middle", dx=5)  # dx=5 untuk memberi jarak antara batang dan teks
    .encode(
        y=alt.Y("METODE_KB:N", sort="-x"),             # Sumbu Y: Metode KB, diurutkan berdasarkan jumlah
        x=alt.X("JUMLAH:Q"),                           # Sumbu X: Jumlah pengguna
        text="JUMLAH:Q"                                # Teks: Nilai jumlah pengguna
    )
)

# Menggabungkan grafik batang dan label
final_chart = chart + text

# Menampilkan grafik
final_chart.show()

alt.LayerChart(...)

In [31]:
import polars as pl
import altair as alt

# Membuat DataFrame Polars
data = pl.DataFrame({
    "PROVINSI": ["SULAWESI BARAT"] * 8,
    "METODE_KB": ["SUNTIK", "PIL", "IMPLAN", "IUD", "TUBEKTOMI", "KONDOM", "MAL", "VASEKTOMI"],
    "JUMLAH": [664, 278, 214, 63, 41, 12, 0, 0]
})

# Menghitung total jumlah pengguna
total_jumlah = data["JUMLAH"].sum()

# Menambahkan kolom persentase (dengan 2 angka desimal)
data = data.with_columns(
    (pl.col("JUMLAH") / total_jumlah * 100).round(2).alias("PERSENTASE")
)

# Mengurutkan data berdasarkan jumlah secara descending
sorted_data = data.sort("JUMLAH", descending=True)

# Konversi ke Pandas DataFrame karena Altair bekerja dengan Pandas
sorted_df = sorted_data.to_pandas()

# Membuat grafik batang horizontal dengan Altair
chart = (
    alt.Chart(sorted_df)
    .mark_bar(color=warna_biru)
    .encode(
        y=alt.Y("METODE_KB:N", title="Metode KB", sort="-x"),  # Sumbu Y: Metode KB, diurutkan berdasarkan jumlah
        x=alt.X("JUMLAH:Q", title="Jumlah Pengguna"),         # Sumbu X: Jumlah pengguna
        tooltip=[
            alt.Tooltip("METODE_KB", title="Metode KB"),
            alt.Tooltip("JUMLAH", title="Jumlah"),
            alt.Tooltip("PERSENTASE", title="Persentase (%)", format=".2f")  # Format 2 angka desimal
        ]
    )
    .properties(
        width=600,                                            # Lebar grafik
        height=300,                                           # Tinggi grafik
        title="Jumlah Pengguna Metode KB di Sulawesi Barat"   # Judul grafik
    )
)

# Menambahkan label di ujung batang
text = (
    alt.Chart(sorted_df)
    .mark_text(align="left", baseline="middle", dx=5)  # dx=5 untuk memberi jarak antara batang dan teks
    .encode(
        y=alt.Y("METODE_KB:N", sort="-x"),             # Sumbu Y: Metode KB, diurutkan berdasarkan jumlah
        x=alt.X("JUMLAH:Q"),                           # Sumbu X: Jumlah pengguna
        text="JUMLAH:Q"                                # Teks: Nilai jumlah pengguna
    )
)

# Menggabungkan grafik batang dan label
final_chart = chart + text

# Menampilkan grafik
final_chart.show()

alt.LayerChart(...)

In [41]:
import polars as pl
import altair as alt

# Membuat DataFrame Polars
data = pl.DataFrame({
    "PROVINSI": ["SULAWESI BARAT"] * 8,
    "METODE_KB": ["SUNTIK", "PIL", "IMPLAN", "IUD", "TUBEKTOMI", "KONDOM", "MAL", "VASEKTOMI"],
    "JUMLAH": [664, 278, 214, 63, 41, 12, 0, 0]
})

# Menghitung total jumlah pengguna
total_jumlah = data["JUMLAH"].sum()

# Menambahkan kolom persentase (dengan 2 angka desimal)
data = data.with_columns(
    (pl.col("JUMLAH") / total_jumlah * 100)
    .round(2)
    .alias("PERSENTASE")
)

# Format kolom persentase ke string dengan koma dan %
persentase_list = data["PERSENTASE"].to_list()
persentase_format = [f"{x:.2f}".replace('.', ',') + '%' for x in persentase_list]
data = data.with_columns(pl.Series("PERSENTASE_FORMAT", persentase_format))

# Mengurutkan data berdasarkan jumlah secara descending
sorted_data = data.sort("JUMLAH", descending=True)

# Konversi ke Pandas DataFrame
sorted_df = sorted_data.to_pandas()

# Membuat grafik batang horizontal dengan Altair
chart = (
    alt.Chart(sorted_df)
    .mark_bar()
    .encode(
        y=alt.Y("METODE_KB:N", title="Metode KB", sort="-x"),
        x=alt.X("JUMLAH:Q", title="Jumlah Pengguna"),
        color=alt.Color("METODE_KB:N", legend=None),
        tooltip=[
            alt.Tooltip("METODE_KB", title="Metode KB"),
            alt.Tooltip("JUMLAH", title="Jumlah"),
            alt.Tooltip("PERSENTASE_FORMAT:N", title="Persentase")
        ]
    )
    .properties(
        width=600,
        height=300,
        title="Jumlah Pengguna Metode KB di Sulawesi Barat"
    )
)

# Menambahkan label di ujung batang
text = (
    alt.Chart(sorted_df)
    .mark_text(align="left", baseline="middle", dx=5)
    .encode(
        y=alt.Y("METODE_KB:N", sort="-x"),
        x=alt.X("JUMLAH:Q"),
        text="JUMLAH:Q"
    )
)

# Menggabungkan grafik batang dan label
final_chart = chart + text

# Menampilkan grafik
final_chart.show()

alt.LayerChart(...)

In [42]:
data

PROVINSI,METODE_KB,JUMLAH,PERSENTASE,PERSENTASE_FORMAT
str,str,i64,f64,str
"""SULAWESI BARAT""","""SUNTIK""",664,52.2,"""52,20%"""
"""SULAWESI BARAT""","""PIL""",278,21.86,"""21,86%"""
"""SULAWESI BARAT""","""IMPLAN""",214,16.82,"""16,82%"""
"""SULAWESI BARAT""","""IUD""",63,4.95,"""4,95%"""
"""SULAWESI BARAT""","""TUBEKTOMI""",41,3.22,"""3,22%"""
"""SULAWESI BARAT""","""KONDOM""",12,0.94,"""0,94%"""
"""SULAWESI BARAT""","""MAL""",0,0.0,"""0,00%"""
"""SULAWESI BARAT""","""VASEKTOMI""",0,0.0,"""0,00%"""


# Tenaga Terlatih

In [45]:
# Data Anda dalam bentuk Polars DataFrame
data = pl.DataFrame({
    "NO": [0, 1, 2, 3, 4],
    "PROVINSI": ["SULAWESI BARAT"] * 5,
    "KABUPATEN": ["PASANGKAYU"] * 5,
    "KECAMATAN": ["SARJO"] * 5,
    "KELURAHAN": ["SARJO"] * 5,
    "NAMA FASKES": ["UPT PUSKESMAS SARJO"] * 5,
    "NO REGISTRASI": ["7601001"] * 5,
    "NAMA BIDAN": ["Mar", "NUR", "Irm", "Rit", "ASM"],
    "NIK BIDAN": [760] * 5,
    "ALAMAT": ["SARJO", "SARUDE", "LETAWA", "LETAWA", "SARUDE"],
    "NO HP": ["NA", "85397882590", "82396497703", "82349677921", "85242892212"],
    "PROFESI": ["Bidan"] * 5,
    "PELATIHAN": [
        "KIP/Konseling,R/R",
        "Belum Terlatih",
        "IUD/IMPLAN,R/R,Sertifikat Kompetensi IUD/Implan",
        "IUD/IMPLAN,KIP/Konseling,R/R",
        "Belum Terlatih"
    ]
})

# Klasifikasi tenaga kerja berdasarkan kolom PELATIHAN
data_with_classification = data.with_columns(
    pl.when(
        pl.col("PELATIHAN").str.contains("(?i)IUD|Implan|Tubektomi|Vasektomi")
    ).then(pl.lit("Sudah Terlatih")).otherwise(pl.lit("Belum Terlatih")).alias("KLASIFIKASI")
)

# Hitung jumlah tenaga kerja untuk setiap klasifikasi
summary_data = data_with_classification.group_by("KLASIFIKASI").agg(
    pl.len().alias("count")  # Menggunakan .len() sebagai pengganti .count()
)

# Tambahkan kolom persentase
total_count = summary_data["count"].sum()
summary_data = summary_data.with_columns(
    (pl.col("count") / total_count * 100).round(2).alias("PERSENTASE")
)

# Format kolom persentase ke string dengan koma dan %
persentase_list = summary_data["PERSENTASE"].to_list()
persentase_format = [f"{x:.2f}".replace('.', ',') + '%' for x in persentase_list]
summary_data = summary_data.with_columns(pl.Series("PERSENTASE_FORMAT", persentase_format))

summary_data

KLASIFIKASI,count,PERSENTASE,PERSENTASE_FORMAT
str,u32,f64,str
"""Belum Terlatih""",3,60.0,"""60,00%"""
"""Sudah Terlatih""",2,40.0,"""40,00%"""


In [46]:
# Definisi warna
color_scale = alt.Scale(
    domain=["Sudah Terlatih", "Belum Terlatih"],
    range=["#1f77b4", "#ff7f0e"]  # Biru untuk Sudah Terlatih, Kuning untuk Belum
)

# Membuat donut chart
base = alt.Chart(summary_data).encode(
    theta=alt.Theta("count:Q", stack=True),
    color=alt.Color("KLASIFIKASI:N", scale=color_scale, legend=None),
    tooltip=[
        alt.Tooltip("KLASIFIKASI:N", title="Status"),
        alt.Tooltip("count:Q", title="Jumlah"),
        alt.Tooltip("PERSENTASE_FORMAT:N", title="Persentase")
    ]
).properties(
    width=300,
    height=300,
    title="Proporsi Klasifikasi Pelatihan"
)

# Arc untuk donut
arc = base.mark_arc(
    outerRadius=100,
    innerRadius=40
).encode(
    text="PERSENTASE_FORMAT:N"  # Untuk label di tengah
)

# Menambahkan label persentase
text = base.mark_text(
    radius=70,  # Posisi label di tengah donut
    fontSize=14,
    align="center",
    baseline="middle"
).encode(
    text=alt.Text("PERSENTASE_FORMAT:N")
)

# Menggabungkan grafik
chart = (arc + text).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=16
).configure(
    background='#f6f8fa'
)

# Menampilkan grafik
chart.show()

alt.LayerChart(...)

In [ ]:
import polars as pl
import altair as alt

# Membuat DataFrame Polars
data = pl.DataFrame({
    "KLASIFIKASI": ["Belum Terlatih", "Sudah Terlatih"],
    "count": [3, 2],
    "PERSENTASE_FORMAT": ["60,00%", "40,00%"]
})

# Konversi ke Pandas DataFrame
df = data.to_pandas()

# Definisi warna
color_scale = alt.Scale(
    domain=["Sudah Terlatih", "Belum Terlatih"],
    range=[warna_biru, warna_kuning]  # Biru dan kuning
)

# Basis chart
base = alt.Chart(df).encode(
    theta=alt.Theta("count:Q", stack=True),
    color=alt.Color("KLASIFIKASI:N", scale=color_scale, legend=None),
    tooltip=[
        alt.Tooltip("KLASIFIKASI:N", title="Status"),
        alt.Tooltip("count:Q", title="Jumlah"),
        alt.Tooltip("PERSENTASE_FORMAT:N", title="Persentase")
    ]
).properties(
    width=500,    # Lebar lebih besar
    height=500,   # Tinggi lebih besar
    title="Proporsi Klasifikasi Pelatihan"
)

# Arc (donut)
arc = base.mark_arc(
    outerRadius=150,  # Radius luar lebih besar
    innerRadius=80    # Radius dalam lebih besar
)

# Label di luar donut dengan penyesuaian posisi
text = base.mark_text(
    radius=200,       # Posisi lebih jauh dari donut (200 > outerRadius 150)
    fontSize=16,
    align="center",
    baseline="middle",
    angle=0,          # Tidak perlu rotasi
    dx=0,             # Offset horizontal
    dy=0              # Offset vertikal
).encode(
    text=[alt.Text("PERSENTASE_FORMAT:N"),
          alt.Text("KLASIFIKASI:N")]
)

# Gabungkan grafik
chart = (arc + text).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=18
).configure(
    background='#f6f8fa'
)

# Tampilkan
chart.show()

alt.LayerChart(...)

In [50]:
import polars as pl
import altair as alt

# Membuat DataFrame Polars
data = pl.DataFrame({
    "KLASIFIKASI": ["Belum Terlatih", "Sudah Terlatih"],
    "count": [3, 2],
    "PERSENTASE": [60.0, 40.0],
    "PERSENTASE_FORMAT": ["60,00%", "40,00%"]
})

# Konversi ke Pandas DataFrame
df = data.to_pandas()

# Definisi warna
color_scale = alt.Scale(
    domain=["Sudah Terlatih", "Belum Terlatih"],
    range=["#1f77b4", "#ff7f0e"]
)

# Membuat donut chart
base = alt.Chart(df).encode(
    theta=alt.Theta("count:Q", stack=True),
    color=alt.Color("KLASIFIKASI:N", scale=color_scale, legend=None),
    tooltip=[
        alt.Tooltip("KLASIFIKASI:N", title="Status"),
        alt.Tooltip("count:Q", title="Jumlah"),
        alt.Tooltip("PERSENTASE_FORMAT:N", title="Persentase")
    ]
).properties(
    width=400,
    height=400,
    title="Proporsi Klasifikasi Pelatihan"
)

# Arc untuk donut
arc = base.mark_arc(
    outerRadius=120,  # Radius donut
    innerRadius=60
)

# Menambahkan label di luar donut
text = base.mark_text(
    radius=140,  # Posisi di luar donut
    fontSize=14,
    align="center",
    baseline="middle",
    angle=alt.value(0)  # Tidak perlu rotasi
).encode(
    text="PERSENTASE_FORMAT:N",
    # Hitung posisi teks berdasarkan theta
    theta=alt.Theta("count:Q", stack=True)
).transform_calculate(
    # Hitung sudut tengah untuk setiap slice
    mid_theta="(datum.theta + datum.theta_end)/2"
).encode(
    angle=alt.Angle("mid_theta:Q")  # Rotasi label sesuai posisi
)

# Menggabungkan grafik
chart = (arc + text).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=16
).configure(
    background='#f6f8fa'
)

# Menampilkan grafik
chart.show()

SchemaValidationError: '{'value': 0}' is an invalid value for `angle`. Valid values are of type 'number'.

Additional properties are not allowed ('value' was unexpected)'expr' is a required property